In [ ]:
!pip install azure-storage-blob # Microsoft Azure
!pip install boto3 # Amazon S3
!pip install google-cloud-storage # Google Cloud Storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 599.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage

In [ ]:
# Example of a local file path
# URL = "/path/to/your/file.parquet"

# Example of a remote URL
# URL = "http://example.com/yourfile.parquet"
URL = "https://data.wa.gov/api/views/f6w7-q2d2/rows.csv?accessType=DOWNLOAD"
# df_raw = pd.read_parquet(URL, engine = 'pyarrow')
df_raw = pd.read_csv(URL)
df_raw = df_raw.drop('Vehicle Location', axis = 1)
print(df_raw.info())
print(df_raw.shape)
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181458 entries, 0 to 181457
Data columns (total 16 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         181458 non-null  object 
 1   County                                             181455 non-null  object 
 2   City                                               181455 non-null  object 
 3   State                                              181458 non-null  object 
 4   Postal Code                                        181455 non-null  float64
 5   Model Year                                         181458 non-null  int64  
 6   Make                                               181458 non-null  object 
 7   Model                                              181458 non-null  object 
 8   Electric Vehicle Type                              181458 non-null  object

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Electric Utility,2020 Census Tract
0,WAUTPBFF4H,King,Seattle,WA,98126.0,2017,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,34.0,235085336,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
1,WAUUPBFF2J,Thurston,Olympia,WA,98502.0,2018,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,22.0,237896795,PUGET SOUND ENERGY INC,5.306701e+10
2,5YJSA1E22H,Thurston,Lacey,WA,98516.0,2017,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,22.0,154498865,PUGET SOUND ENERGY INC,5.306701e+10
3,1C4JJXP62M,Thurston,Tenino,WA,98589.0,2021,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,20.0,154525493,PUGET SOUND ENERGY INC,5.306701e+10
4,5YJ3E1EC9L,Yakima,Yakima,WA,98902.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,14.0,225996361,PACIFICORP,5.307700e+10


In [ ]:
#Connection String
CONNECTION_STRING_AZURE_STORAGE = "DefaultEndpointsProtocol=https;AccountName=jwrcis4400spring2024;AccountKey=0571bLyinef27rYbGGUROa4Djku3+vMH4ER2stcp2APdZoGOr4D1AMFnvgJOXRXihFOItHSNn1mW+AStHAbrwA==;EndpointSuffix=core.windows.net"
#Container for bucket
CONTAINER_AZURE = 'evpopulationdata'
blob_name = "Electric_Vehicle_Population_Data.csv"
#Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

#create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

#Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

#Upload the CSV data
blob_client.upload_blob(data, overwrite = True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")


Uploaded Electric_Vehicle_Population_Data.csv to Azure Blob Storage in container evpopulationdata.


In [ ]:
# Functions

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
  blob_service_client = BlobServiceClient.from_connection_string(connect_str)
  blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
  blob_client.upload_blob(data, overwrite=True)
  print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
  blob_service_clinet = BlobServiceClient.from_connection_string(connect_str)
  blob_client = blob_service_client.get_blkob_cliient(container=container_name, blob=blob_name)
  download_stream = blob_client.download_blob()
  return download_stream.readall()

#Google cloud Functions
def google_upload_blob(butcket_name, source_file_name, destination_blob_name):
  storage_client = storage.Client()
  bucket = storage_client.bucket(bucket_name)
  blob = bucket.blob(destination_blob_name)
  blob.upload_from_filename(source_file_name)
  print(f"Fiel {source_file_name} uploaded to {destination_blob_name}.")

In [ ]:
DefaultEndpointsProtocol=https;AccountName=jwrcis4400spring2024;AccountKey=0571bLyinef27rYbGGUROa4Djku3+vMH4ER2stcp2APdZoGOr4D1AMFnvgJOXRXihFOItHSNn1mW+AStHAbrwA==;EndpointSuffix=core.windows.net